# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  May 27 2022 12:00AM  238212        10    CTF0011253   
1  May 27 2022 12:00AM  238212        10    CTF0011254   
2  May 27 2022 12:00AM  238214        10  SONSB0001801   
3  May 27 2022 12:00AM  238216        10     MET210663   
4  May 26 2022  5:43PM  238225        16       8052464   
5  May 26 2022  5:43PM  238225        16       8054316   
6  May 26 2022  5:43PM  238225        16       8056945   
7  May 26 2022  5:43PM  238225        16       8057002   
8  May 26 2022  5:43PM  238225        16       8057043   
9  May 26 2022  5:43PM  238225        16       8056957   

                         Customer ShipmentStatus  
0                  Citieffe, Inc.       Released  
1                  Citieffe, Inc.       Released  
2   Nextsource Biotechnology, LLC       Released  
3                Methapharm, Inc.       Released  
4  Sartorius Bioprocess Solutions       Released  
5  Sartorius Bioprocess Solutions       Released  
6  Sartorius Bioprocess Solutions       Released  
7  Sartorius Bioprocess Solutions       Released  
8  Sartorius Bioprocess Solutions       Released  
9  Sartorius Bioprocess Solutions       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
31  238214       Released          1
32  238215       Released          1
33  238216       Released          1
34  238219       Released          1
35  238225       Released          9

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238214                NaN        NaN       1.0
238215                NaN        NaN       1.0
238216                NaN        NaN       1.0
238219                NaN        NaN       1.0
238225                NaN        NaN       9.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238129                0.0        0.0       1.0
238133                6.0        2.0       3.0
238139               19.0        2.0       1.0
238153                0.0        0.0       1.0
238159                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238129                  0          0         1
238133                  6          2         3
238139                 19          2         1
238153                  0          0         1
238159                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238129          0          0         1
1               238133          6          2         3
2               238139         19          2         1
3               238153          0          0         1
4               238159          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               238129                             1
1               238133         6         2         3
2               238139        19         2         1
3               238153                             1
4               238159                             1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   May 27 2022 12:00AM  238212        10                  Citieffe, Inc.
2   May 27 2022 12:00AM  238214        10   Nextsource Biotechnology, LLC
3   May 27 2022 12:00AM  238216        10                Methapharm, Inc.
4   May 26 2022  5:43PM  238225        16  Sartorius Bioprocess Solutions
13  May 26 2022  5:00PM  238219        16                  HealthCaps LLC
14  May 26 2022  3:45PM  238210        12             LF of America Corp.
15  May 26 2022  3:29PM  238204        19          GCH Granules USA, Inc.
16  May 26 2022  3:19PM  238215        22               NBTY Global, Inc.
17  May 26 2022  3:00PM  238200        16            Emersa Waterbox, LLC
18  May 26 2022  2:54PM  238199        19           ACG North America LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  May 27 2022 12:00AM  238212        10                  Citieffe, Inc.   
1  May 27 2022 12:00AM  238214        10   Nextsource Biotechnology, LLC   
2  May 27 2022 12:00AM  238216        10                Methapharm, Inc.   
3  May 26 2022  5:43PM  238225        16  Sartorius Bioprocess Solutions   
4  May 26 2022  5:00PM  238219        16                  HealthCaps LLC   
5  May 26 2022  3:45PM  238210        12             LF of America Corp.   
6  May 26 2022  3:29PM  238204        19          GCH Granules USA, Inc.   
7  May 26 2022  3:19PM  238215        22               NBTY Global, Inc.   
8  May 26 2022  3:00PM  238200        16            Emersa Waterbox, LLC   
9  May 26 2022  2:54PM  238199        19           ACG North America LLC   

  Completed Executing  Released  
0                             2  
1                             1  
2                             1  
3                             9  
4                             1  
5                             1  
6                             1  
7                             1  
8                             1  
9                             1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  May 27 2022 12:00AM  238212        10                  Citieffe, Inc.   
1  May 27 2022 12:00AM  238214        10   Nextsource Biotechnology, LLC   
2  May 27 2022 12:00AM  238216        10                Methapharm, Inc.   
3  May 26 2022  5:43PM  238225        16  Sartorius Bioprocess Solutions   
4  May 26 2022  5:00PM  238219        16                  HealthCaps LLC   
5  May 26 2022  3:45PM  238210        12             LF of America Corp.   
6  May 26 2022  3:29PM  238204        19          GCH Granules USA, Inc.   
7  May 26 2022  3:19PM  238215        22               NBTY Global, Inc.   
8  May 26 2022  3:00PM  238200        16            Emersa Waterbox, LLC   
9  May 26 2022  2:54PM  238199        19           ACG North America LLC   

   Released Executing Completed  
0         2                      
1         1                      
2         1                      
3         9                      
4         1                      
5         1                      
6         1                      
7         1                      
8         1                      
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  May 27 2022 12:00AM  238212        10                  Citieffe, Inc.   
1  May 27 2022 12:00AM  238214        10   Nextsource Biotechnology, LLC   
2  May 27 2022 12:00AM  238216        10                Methapharm, Inc.   
3  May 26 2022  5:43PM  238225        16  Sartorius Bioprocess Solutions   
4  May 26 2022  5:00PM  238219        16                  HealthCaps LLC   

   Released Executing Completed  
0         2                      
1         1                      
2         1                      
3         9                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  May 27 2022 12:00AM  238212        10                  Citieffe, Inc.   
1  May 27 2022 12:00AM  238214        10   Nextsource Biotechnology, LLC   
2  May 27 2022 12:00AM  238216        10                Methapharm, Inc.   
3  May 26 2022  5:43PM  238225        16  Sartorius Bioprocess Solutions   
4  May 26 2022  5:00PM  238219        16                  HealthCaps LLC   

   Released  Executing  Completed  
0         2        NaN        NaN  
1         1        NaN        NaN  
2         1        NaN        NaN  
3         9        NaN        NaN  
4         1        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  May 27 2022 12:00AM  238212        10                  Citieffe, Inc.   
1  May 27 2022 12:00AM  238214        10   Nextsource Biotechnology, LLC   
2  May 27 2022 12:00AM  238216        10                Methapharm, Inc.   
3  May 26 2022  5:43PM  238225        16  Sartorius Bioprocess Solutions   
4  May 26 2022  5:00PM  238219        16                  HealthCaps LLC   

   Released  Executing  Completed  
0         2        0.0        0.0  
1         1        0.0        0.0  
2         1        0.0        0.0  
3         9        0.0        0.0  
4         1        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1905531        33        0.0        0.0
12          238210         1        0.0        0.0
15          238139         1        2.0       19.0
16         1429132        14        0.0        0.0
18          476292         4        2.0        6.0
19          952768         5        2.0        0.0
20         1190854        35        1.0        2.0
21          238163         1        0.0        0.0
22          238215         1        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1905531        33        0.0        0.0
1        12   238210         1        0.0        0.0
2        15   238139         1        2.0       19.0
3        16  1429132        14        0.0        0.0
4        18   476292         4        2.0        6.0
5        19   952768         5        2.0        0.0
6        20  1190854        35        1.0        2.0
7        21   238163         1        0.0        0.0
8        22   238215         1        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        33        0.0        0.0
1        12         1        0.0        0.0
2        15         1        2.0       19.0
3        16        14        0.0        0.0
4        18         4        2.0        6.0
5        19         5        2.0        0.0
6        20        35        1.0        2.0
7        21         1        0.0        0.0
8        22         1        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   33.0
1        12  Released    1.0
2        15  Released    1.0
3        16  Released   14.0
4        18  Released    4.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15    16   18   19    20   21   22
Status                                                    
Completed   0.0  0.0  19.0   0.0  6.0  0.0   2.0  0.0  0.0
Executing   0.0  0.0   2.0   0.0  2.0  2.0   1.0  0.0  0.0
Released   33.0  1.0   1.0  14.0  4.0  5.0  35.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15    16   18   19    20   21   22
0          Completed   0.0  0.0  19.0   0.0  6.0  0.0   2.0  0.0  0.0
1          Executing   0.0  0.0   2.0   0.0  2.0  2.0   1.0  0.0  0.0
2           Released  33.0  1.0   1.0  14.0  4.0  5.0  35.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15    16   18   19    20   21   22
0  Completed   0.0  0.0  19.0   0.0  6.0  0.0   2.0  0.0  0.0
1  Executing   0.0  0.0   2.0   0.0  2.0  2.0   1.0  0.0  0.0
2   Released  33.0  1.0   1.0  14.0  4.0  5.0  35.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()